In [165]:
from twelvedata import TDClient
import os
import pandas as pd

In [166]:
td = TDClient(os.getenv('TD_API'))

In [167]:
all_symbols = ("AUD/CAD","AUD/CHF","AUD/JPY","AUD/NZD","AUD/USD",
               "CAD/CHF", "CAD/JPY", "CHF/JPY", 
               "EUR/AUD", "EUR/CAD", "EUR/CHF", "EUR/GBP", "EUR/JPY", "EUR/NZD", "EUR/USD",
               "GBP/AUD", "GBP/CAD", "GBP/CHF", "GBP/JPY", "GBP/NZD", "GBP/USD",
               "NZD/CAD", "NZD/CHF", "NZD/JPY", "NZD/USD", 
               "USD/CAD", "USD/CHF", "USD/JPY")

In [168]:
portfolio_sheet = pd.read_csv('portfolio_example.csv')

In [169]:
for pair in portfolio_sheet['pair']:
    if pair in all_symbols:
      continue
    else:
      print("Symbol is not allowed or mispelled.  Please check the 'all_symbols' variable.")
      exit()

In [174]:
table = pd.DataFrame()

for _symbol in portfolio_sheet['pair']:
  ts = td.time_series(
    symbol = _symbol,
    interval="1day",
    outputsize="14"
  ).as_pandas()

  ts.drop(['open', 'high', 'low'], axis=1, inplace=True)
  ts.rename(columns={'close': str(_symbol)}, inplace=True)
  table = pd.concat([table, ts], axis=1)


table = table.pct_change()
print(table)
print(table.corr())

             GBP/USD   EUR/USD   USD/JPY   AUD/CHF   AUD/NZD
datetime                                                    
2023-08-29       NaN       NaN       NaN       NaN       NaN
2023-08-28 -0.001545 -0.003225  0.003802  0.000176  0.001565
2023-08-25 -0.001786 -0.002218 -0.000580 -0.003168 -0.002970
2023-08-24  0.001828  0.001390 -0.004233  0.001942 -0.001079
2023-08-23  0.009800  0.004903 -0.006789  0.002819  0.000554
2023-08-22  0.000550 -0.001657  0.007215 -0.006308 -0.003607
2023-08-21  0.001846  0.004703  0.002228 -0.003625  0.001907
2023-08-18 -0.001489 -0.002203 -0.005608  0.002999 -0.000508
2023-08-17  0.000746  0.000000  0.003026 -0.004583 -0.000832
2023-08-16 -0.001216  0.000690  0.003497  0.004871  0.001388
2023-08-15 -0.002121  0.002252 -0.005227  0.003007  0.002310
2023-08-14 -0.001574  0.000183 -0.000172  0.004938  0.001106
2023-08-11  0.000946  0.003668 -0.004088 -0.000983 -0.000470
2023-08-10 -0.001654  0.003289 -0.001414  0.003531 -0.003031
          GBP/USD   EUR/

In [171]:
pivot_risk = portfolio_sheet.iloc[0,1]
drawdown = pivot_risk

In [172]:
pair_counter = 1
for i in range(((len(portfolio_sheet) -1))):
  corr = table.iloc[:,0].corr(table.iloc[:,pair_counter])
  pair_risk = portfolio_sheet.iloc[pair_counter, 1]
  add_drawdown = corr * pair_risk
  drawdown += add_drawdown
  if pair_counter < len(portfolio_sheet) -1:
    pair_counter += 1

print("Expected drawdown is: %", -1 * round(drawdown, 2))

Expected drawdown is: % -1.07
